# Sequential Physics Based Rigid Body Stacking

In [1]:
import pybullet as p
import os 
import time
import pybullet_data
import threading
import numpy as np
import scipy as sp
import trimesh

In [2]:
def get_rock_filepaths(): 
    filenames = os.listdir("./Rocks/SingleRockFiles_Raw")
    return [f'./Rocks/SingleRockFiles_Raw/{fname}' for fname in filenames if '.obj' in fname.lower()]

In [3]:
#Setup the physics engine. 
physicsClient = p.connect(p.GUI) #or p.DIRECT for non-graphical version
p.setAdditionalSearchPath(pybullet_data.getDataPath()) #optionally
p.setGravity(0,0,-5) #world has no gravity but we add it to objects when performing stacking

#Create a platform upon which we will perform stacking 
planeId = p.loadURDF("plane.urdf", basePosition=[0,0,0])

In [4]:
#the filepaths to our rock objects (.obj files)
rockfiles = get_rock_filepaths() 

#the number of rocks total 
nrocks = len(rockfiles)

#the number of rocks we place in the scene 
use_nrocks = 5

#select some randomized subset of rocks
#these are the rocks we will place in the physics simulator 
rockindex = [i for i in range(nrocks)]
np.random.seed(42)
np.random.shuffle(rockindex)
rockfiles = [rockfiles[i] for i in rockindex[:use_nrocks]]

In [ ]:
def create_rock_in_simulation(rfile): 
    
    meshScale = [.04, .04, .04]
    
    visualShapeId = p.createVisualShape(shapeType=p.GEOM_MESH, 
                                        fileName=rfile,  
                                        meshScale=meshScale
                                       )

    collisionShapeId = p.createCollisionShape(shapeType=p.GEOM_MESH, 
                                              fileName=rfile, 
                                              meshScale=meshScale
                                             )

    rockid = p.createMultiBody(baseCollisionShapeIndex=collisionShapeId, 
                              baseVisualShapeIndex=visualShapeId,
                              baseMass=5)
    
    return rockid 

In [ ]:
def get_object_bounding_box(obj_id): 
    #we are interested in the maximal bounding box of an object 
    aabbMin, aabbMax = p.getAABB(obj_id)
    return aabbMin, aabbMax 

def get_xyz_com_from_aabb(aabbMin, aabbMax): 
    '''
    Currently, we are approximating a 3D axis aligned 
    bounding box and computing the centroid. 
    '''
    return [
        (aabbMin[0]+aabbMax[0])/2,
        (aabbMin[1]+aabbMax[1])/2,
        (aabbMin[2]+aabbMax[2])/2,
    ]

class TowerOfRigidBodies: 
    
    def __init__(self, tower_com):
        #The center of mass of the existing tower 
        #The tower grows in the z direction so the COM 
        #is a 2-dimensioal (x,y) coordinate 
        self.tower_com = tower_com
        
        #The largest z value for any vertex contained within the whole tower
        #When stacking a new object on top of the tower, we transform the object
        #from its starting position to a point directly above the COM of the tower
        #such that the vertex in the object to be stacked with the smallest z value 
        #is resting atop the 2D plane at this z value 
        self.tower_bb_top_z = 0
        
        #The collection of object ids that defines the order in which objects
        #were stacked to create the current tower 
        self.object_ids = []
        
        #The collection of (position,orientation) pairs
        #which define the 3-dimensional paramaters of the 
        #object as it exists in the tower 
        self.object_states = []
        
        #NOTE: len(object_ids) == len(object_states)
        
    def stack(obj_id): 
        to_stack_bb = get_object_bounding_box(obj_id)
        
          

In [ ]:
#Place the rocks in the simulation 
rock_ids = []
for rfile in rockfiles: 
    print(rfile)
    rock_id = create_rock_in_simulation(rfile)
    rock_ids.append(rock_id)

./Rocks/SingleRockFiles_Raw/rock8.obj
./Rocks/SingleRockFiles_Raw/rock13.obj
./Rocks/SingleRockFiles_Raw/rock19.obj
./Rocks/SingleRockFiles_Raw/rock0.obj
./Rocks/SingleRockFiles_Raw/rock28.obj


In [ ]:
while 1: 
    p.stepSimulation()
    time.sleep(1./240.)